In [1]:
#### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

print(sys.version)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/suyambhoo.v4.0.0019/pytorch/4.0.0019/1/best_model.4.0.0019chkpt
/kaggle/input/arc-prize-2025/arc-agi_training_solutions.json
/kaggle/input/arc-prize-2025/arc-agi_evaluation_solutions.json
/kaggle/input/arc-prize-2025/arc-agi_evaluation_challenges.json
/kaggle/input/arc-prize-2025/sample_submission.json
/kaggle/input/arc-prize-2025/arc-agi_training_challenges.json
/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json
3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]


In [2]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from pprint import pprint

NUM_OF_EPOCHS = 4000

def load_arc_data(json_path):
    """Load ARC AGI data from JSON file"""
    with open(json_path, 'r') as f:
        data = json.load(f)

    training_pairs = {}  # Dict of problem_id and corresponding (input_tensor, output_tensor)
    testing_inputs = {}  # Dict of problem_id and corresponding input_tensors

    first = True
    first_problem_id = None
    
    for problem_id, problem in data.items():
        # Train data
        for pair in problem['train']:
            if first:
                first_problem_id = problem_id
                # print("First Problem ID:", first_problem_id)
                # print("First Pair:")
                # pprint(pair)
            inp = pair['input']
            out = pair['output']
            if first:
                first = False
            training_pairs[problem_id] = (inp, out)
            if first:
                pprint(training_pairs)
        # Test data
        for test_case in problem['test']:
            inp = test_case['input']
            testing_inputs[problem_id] = inp

    return first_problem_id, training_pairs, testing_inputs

def build_output(input, input_like, output_like):
    input = np.array(input)
    output_like = np.array(output_like)
    b = output_like == input_like[0][0]
    pred = np.zeros_like(output_like)
    if input.shape == output_like.shape:
        print("Now#1!")
        for i in range(1, 10):
            c = output_like == i
            if np.any(c):
                print("Now#2!", i)
                for j in range(1, 10):
                    if j == i:
                        continue
                    print("Now#3!", i, j)
                    cc = output_like == j
                pred = np.nan
                try:
                    pprint(input_like)
                    pprint(c)
                    d = input_like[c]
                    pprint(d)
                    d = d[0][0]
                    dd = input_like[cc]
                    pprint(dd)
                    dd = dd[0][0]
                    p = input == d
                    pp = input == dd
                    pred = input.copy()
                    pred[p] = i
                    pred[pp] = j
                except Exception as e:
                    print("Exception...", e)
                return pred
    else:
        for index, element in np.ndenumerate(pred):
            if b[index]:
                try:
                    if ((index[0] < output_like.shape[0] - 1 and \
                        output_like[index[0]+1,index[1]] == input_like[0][1]) or \
                        (output_like[index[0],index[1]] == input_like[0][1])) and \
                        ((index[1] < output_like.shape[1] - 1 and \
                        output_like[index[0],index[1]+1] == input_like[1][0]) or \
                        (output_like[index[0],0] == input_like[1][0])):
                            if pred[index[0]:index[0]+input.shape[0], index[1]:index[1]+input.shape[1]].shape == input.shape:
                                pred[index[0]:index[0]+input.shape[0], index[1]:index[1]+input.shape[1]] = input
                            else:
                                if not (index[1] < output_like.shape[1] - 1):
                                    pre_val = pred[index[0]:index[0]+input.shape[0], index[1]:index[1]+input.shape[1]]
                                    pred[index[0]:index[0]+input.shape[0], index[1]:index[1]+input.shape[1]] = input[0:pre_val.shape[1],].reshape(2,1)
                                    pred[index[0]:index[0]+input.shape[0],0:input.shape[1]-pre_val.shape[1]] = input[pre_val.shape[1]:pre_val.shape[1]+1,].reshape(2,1)
                except Exception as e:
                    print("Exception!", e)
                    pass
    return pred

def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Load data
    print("Loading ARC AGI data...")
    first_problem_id, training_pairs, testing_inputs = load_arc_data('/kaggle/input/arc-prize-2025/arc-agi_training_challenges.json')
    print(f"Loaded {len(training_pairs)} training pairs and {len(testing_inputs)} test inputs.")
    for key, val in training_pairs.items():
        print(key)
        input = testing_inputs[key]
        pprint(input)
        pprint(val[0])
        pprint(val[1])
        # break
        # pred = build_output(input, val[0], val[1])
        # # if np.array_equal(pred, val[1]):
        # print("#"*40)
        # print("Success!")
        # pprint(pred)
        # print("#"*40)

if __name__ == "__main__":
    main()

Using device: cpu
Loading ARC AGI data...
Loaded 1000 training pairs and 1000 test inputs.
00576224
[[3, 2], [7, 8]]
[[8, 6], [6, 4]]
[[8, 6, 8, 6, 8, 6],
 [6, 4, 6, 4, 6, 4],
 [6, 8, 6, 8, 6, 8],
 [4, 6, 4, 6, 4, 6],
 [8, 6, 8, 6, 8, 6],
 [6, 4, 6, 4, 6, 4]]
007bbfb7
[[7, 0, 7], [7, 0, 7], [7, 7, 0]]
[[0, 7, 7], [7, 7, 7], [0, 7, 7]]
[[0, 0, 0, 0, 7, 7, 0, 7, 7],
 [0, 0, 0, 7, 7, 7, 7, 7, 7],
 [0, 0, 0, 0, 7, 7, 0, 7, 7],
 [0, 7, 7, 0, 7, 7, 0, 7, 7],
 [7, 7, 7, 7, 7, 7, 7, 7, 7],
 [0, 7, 7, 0, 7, 7, 0, 7, 7],
 [0, 0, 0, 0, 7, 7, 0, 7, 7],
 [0, 0, 0, 7, 7, 7, 7, 7, 7],
 [0, 0, 0, 0, 7, 7, 0, 7, 7]]
009d5c81
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8],
 [0, 0, 0, 0, 0, 8, 0, 0, 0, 8, 0, 8, 0, 8],
 [0, 0, 0, 0, 0, 8, 0, 8, 0, 8, 0, 0, 0, 8],
 [0, 0, 0, 0, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1